In [1]:
import pandas as pd
from atlassian import Jira
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()
import xlwings as xw  


In [2]:
url = "https://defencedigital.atlassian.net"

project_folder = os.path.expanduser('~/Dev/python/jira-extractor')  # adjust as appropriate
load_dotenv(os.path.join(project_folder, '.env'))
JIRA_USER = os.getenv('JIRA_USER')
JIRA_PW = os.getenv('JIRA_PW')
PROJECT = 'Ballistic'

jira_instance = Jira(
    url = url,
    username = JIRA_USER,
    password = JIRA_PW,
)

print('JIRA_PW: ' + JIRA_PW);

# Get results of jql query
results = jira_instance.jql("'Product Team' = {}".format(PROJECT), limit = 1000, fields=["resolutiondate", "created", "issuetype", "key", "summary"])


# Load the results into a DataFrame
df = pd.json_normalize(results["issues"])

# Define which fields we care about using dot notation for nested fields
FIELDS_OF_INTEREST = ["fields.resolutiondate", "fields.created", "fields.issuetype.name", "key", "fields.summary"]

# Filter to only display the fields we care about. To actually filter them out use df = df[FIELDS_OF_INTEREST].
df = df[FIELDS_OF_INTEREST]

print(df)

df['fields.resolutiondate'] = pd.to_datetime(df['fields.resolutiondate'], dayfirst = True)
df['fields.created'] = pd.to_datetime(df['fields.created'], dayfirst = True)

date = datetime.now().strftime("%d%m%y")

PROJECT_NO_SPACE = PROJECT.replace(" ", "")
PROJECT_NO_SPACE_NO_BRACE = PROJECT_NO_SPACE.replace('"','')
print(PROJECT_NO_SPACE)
print(PROJECT_NO_SPACE_NO_BRACE)

path = r'{}/reports/{}/Monte-Carlo-Team-Dashboard-raf-{}-{}.xlsx'.format(project_folder,PROJECT_NO_SPACE_NO_BRACE,PROJECT_NO_SPACE_NO_BRACE,date)

book = xw.Book(path)
sheet=book.sheets['Your Data']

cell_row_number = 2
cell_column_number = 1
sheet.cells(cell_row_number,cell_column_number).options(index=False, header=False).value = df

book.save(path)


JIRA_PW: GVmbyWB6XXdCiJQ93oOL6842
   fields.resolutiondate                fields.created fields.issuetype.name  \
0                   None  2023-03-15T19:17:11.514+0000                   Bug   
1                   None  2023-03-15T13:24:30.206+0000                  Task   
2                   None  2023-03-15T13:18:45.518+0000                Design   
3                   None  2023-03-15T13:18:16.339+0000                 Story   
4                   None  2023-03-15T13:06:44.632+0000                 Story   
..                   ...                           ...                   ...   
95                  None  2023-01-26T15:09:30.541+0000                 Story   
96                  None  2023-01-26T15:03:23.928+0000                 Story   
97                  None  2023-01-26T15:00:02.507+0000                 Story   
98                  None  2023-01-26T14:44:05.943+0000                 Story   
99                  None  2023-01-26T14:40:58.534+0000                 Story   

     